In [1]:
from shapely.geometry import asPoint
import geopandas as gpd
import pandas as pd
import time
import shapely
import matplotlib.pyplot as plt


In [2]:
def VBOCleaner(data):
    print(data.shape)
    
    # only keep objects with id
    data = data[data["identificatie"].notna()]
    print(data.shape)
    
    # only keep objects which are currently in use # CHECK of dit de juiste zijn
    data = data[data.verblijfsobjectStatus.isin(["Verblijfsobject gevormd", 
                                                 "Verblijfsobject in gebruik",
                                                 "Verblijfsobject in gebruik (niet ingemeten)"])]
    data = data[data.pos.notna()]
    print(data.shape)
    
    data["begindatumTijdvakGeldigheid"] =  pd.to_datetime(data['begindatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    data["einddatumTijdvakGeldigheid"] =  pd.to_datetime(data['einddatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    
    def toPoint(pos):
        return asPoint([float(i) for i in pos.split()[:2]])
    data["pos"] = data.pos.apply(toPoint)
    
    return data

In [3]:
vbo = pd.read_csv("VBO.csv", sep = ";", encoding = "utf-8", na_values = "NaN", on_bad_lines = "warn")
vbo = VBOCleaner(vbo)


(19501545, 7)
(18918593, 7)
(18386142, 7)


In [4]:
grid = pd.read_csv("Eindhoven_grid.csv")
grid['geometry'] = grid.geometry.apply(shapely.wkt.loads)
grid = gpd.GeoDataFrame(grid)

In [5]:
points = gpd.GeoDataFrame(vbo, geometry = "pos")


In [8]:
test = points[15000000:]

In [9]:
"""
TODO:: make PointInPolys only return C2899R100 and identificatie.
       -->> Make pretty. causes problems now
"""

# Only keep vbo_id and the position of the vbo
vbo = vbo[["identificatie", "pos"]]

t0 = time.time()
# points = gpd.GeoDataFrame(vbo, geometry = "pos")
# del(vbo) # del vbo since it's put into points df. makes space

# join each cbs row with a vbo that contains a vbo point
pointInPolys = gpd.tools.sjoin(test, grid, op = "within")

print(time.time() - t0)

230.25907111167908


In [10]:
pointInPolys

,gebruiksdoelVerblijfsobject,oppervlakteVerblijfsobject,verblijfsobjectStatus,begindatumTijdvakGeldigheid,einddatumTijdvakGeldigheid,identificatie,pos,index_right,Unnamed: 0,C28992R100
17471161,woonfunctie,112.0,Verblijfsobject in gebruik,1988-12-02 00:00:00.010,NaT,1.658100e+15,POINT (165724.227 379403.873),14034,3403832,E1657N3794
17471163,sportfunctie,999999.0,Verblijfsobject in gebruik,2010-09-21 00:00:00.010,2011-04-28 01:00:00.020,1.658100e+15,POINT (165724.227 379403.873),14034,3403832,E1657N3794
18602626,woonfunctie,156.0,Verblijfsobject in gebruik,2013-03-21 00:00:00.000,NaT,1.771100e+15,POINT (166206.606 380747.945),12284,3387417,E1662N3807
18602802,woonfunctie,140.0,Verblijfsobject in gebruik (niet ingemeten),2020-12-03 00:00:01.000,NaT,1.771100e+15,POINT (166242.233 380777.480),12284,3387417,E1662N3807
18603175,woonfunctie,197.0,Verblijfsobject in gebruik,1933-11-03 00:00:00.000,NaT,1.771100e+15,POINT (166219.359 380750.394),12284,3387417,E1662N3807
...,...,...,...,...,...,...,...,...,...,...
18634323,woonfunctie,99.0,Verblijfsobject gevormd,2020-06-05 00:00:01.000,NaT,1.771100e+15,POINT (166258.575 381268.205),11609,3380914,E1662N3812
18634324,woonfunctie,99.0,Verblijfsobject gevormd,2020-06-05 00:00:01.000,NaT,1.771100e+15,POINT (166262.731 381274.688),11609,3380914,E1662N3812
18634396,woonfunctie,128.0,Verblijfsobject gevormd,2020-08-18 00:00:01.000,NaT,1.771100e+15,POINT (165345.705 381519.967),11195,3377009,E1653N3815
18634399,kantoorfunctie,977.0,Verblijfsobject gevormd,2020-08-28 00:00:01.000,2021-05-04 00:00:02.000,1.771100e+15,POINT (165345.705 381519.967),11195,3377009,E1653N3815


In [47]:
pointInPolys.to_csv("eindhoven_vbo.csv")

In [ ]:
# plot example

plt.plot(*cbs[cbs["C28992R100"] == pointInPolys.iloc[0].C28992R100].geometry.iloc[0].exterior.xy)
plt.scatter(point.x, point.y)